In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, BasicAer, execute
from exp_a4 import A4Factory
from exp_a23 import A23Factory
import numpy as np
from qiskit.tools.visualization import circuit_drawer
from scipy.linalg import expm
%matplotlib inline

In [2]:
# number of qubits n, where size of the matrix is 2**n
n = 2
t = 1
# b entry of the matrices
b_4 = 1/3
b_2 = 1/3
# Number of elements in the sum
n_sum = 1
# number of ancilla qubits
n_ancilla = n-1

In [3]:
# TODO: for sums with more than 2 elements need to change qr[n] by the right index
#quantum register for the matrix
qr = QuantumRegister(n)
qr_sum = QuantumRegister(n_sum)
#ancilla qubits
qra = QuantumRegister(n_ancilla)
cr = ClassicalRegister(n+n_sum+n_ancilla)
qc = QuantumCircuit(qr, qr_sum, qra, cr)
# TODO: substitute H by V
qc.h(qr_sum[0])

# TODO: substitute by suitable exp(iH_jt)

#1 controlled H_1 (here is exp_a4)
A4Factory(n).build_controlled(qc, qr, qr_sum[0], qra, b_4*t)
#0 controlled H_2 (here is exp_a23)
qc.x(qr_sum[0])
A23Factory(n).build_controlled(qc, qr, qr_sum[0], qra, b_2*t)
qc.x(qr_sum[0])

# TODO: substitute H by V^dagger
qc.h(qr_sum[0])

In [28]:
# TODO: add some measurement
# Obtain vector from the circuit
# Measure
backend = BasicAer.get_backend('statevector_simulator')
# Create a Quantum Program for execution 
job = execute(qc, backend)
# and store the result
result = job.result()
# Remember its an...a0qn..q0, where the a's are the ancilla
svect = result.get_statevector(qc)

# total number of qubits in the circuit
lastq = n+n_sum+n_ancilla

formatstr = "{0:0"+str(lastq)+"b}"
# Final vector (i.e. approximate solution of e^iHt |x>)
approx_sol = [0]*(2**n)

totalnorm= 0
for i in range(0, len(svect)):
    # Only consider the entries where the sum qubits are 0 (i.e. successful circuit)
    if formatstr.format(i)[0:n_ancilla+n_sum] == '0'*(n_ancilla+n_sum):
                totalnorm += np.real(svect[i]*np.conjugate(svect[i]))
                current_index = int(formatstr.format(i)[lastq-n:lastq][::-1], 2)
                approx_sol[current_index] += svect[i]
                
approx_sol = approx_sol/np.sqrt(totalnorm)

In [29]:
# analytical case
H23 = np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), -1) + np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), 1)
H4 = np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), -1) + np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), 1)

exact_exp = expm(1j*(H23+H4)*t)

test = np.dot(exact_exp, np.append(1,[0]*((2**n)-1)))
print(test/np.linalg.norm(test))
print(approx_sol)

[ 0.94546377+0.j          0.        +0.32115795j -0.05402751+0.j
  0.        -0.00607068j]
[0.99652979+0.j         0.        +0.j         0.        -0.08323692j
 0.        +0.j        ]


In [27]:
print(svect)

[0.99307162+0.j         0.        -0.08294807j 0.        +0.j
 0.        +0.j         0.00692838+0.j         0.        +0.08294807j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j        ]
